In [3]:
%cd /kaggle/working

/kaggle/working


In [4]:
import datetime
import gc
from glob import glob
from pathlib import Path

import netCDF4  # implicitly required by xarray to load .nc
import numpy as np
import pandas as pd
import polars as pl
import tqdm
import xarray as xr
from tqdm import tqdm

In [5]:
paths = sorted(glob("input/ClimSim_low-res/train/0002-*/*.npy"))

In [6]:
len(paths), paths[:1]

(26280, ['input/ClimSim_low-res/train/0002-01/0002-01-01-00000.npy'])

mappingに欲しい情報

- key
  - pbuf_ozone_2
- value
  - location
  - timestamp

In [39]:
pbuf_ozone_list = []
month_list = []
day_list = []
seconds_list = []
skip = 10
for i, path in enumerate(tqdm(paths)):
    x = np.load(path)
    pbuf_ozone_list.append(x[:, 377:437:skip].copy())
    name = Path(path).name
    timestamp_str = name.split(".")[0]
    year, month, day, seconds = timestamp_str.split("-")
    year, month, day, seconds = int(year), int(month), int(day), int(seconds)
    month_list.append([month] * 384)
    day_list.append([day] * 384)
    seconds_list.append([seconds] * 384)
    break

pbuf_ozone_list = np.vstack(pbuf_ozone_list)
month_list = [item for sublist in month_list for item in sublist]
day_list = [item for sublist in day_list for item in sublist]
seconds_list = [item for sublist in seconds_list for item in sublist]

  0%|          | 0/26280 [00:00<?, ?it/s]


In [41]:
map_df = (
    pl.concat(
        [
            pl.from_numpy(
                pbuf_ozone_list,
                schema=[f"pbuf_ozone_{skip*s}" for s in range(60 // skip)],
            ),
            pl.DataFrame(
                [
                    pl.Series(name="month", values=month_list),
                    pl.Series(name="day", values=day_list),
                    pl.Series(name="seconds", values=seconds_list),
                ]
            ),
        ],
        how="horizontal",
    )
    .with_columns((pl.col("seconds") // (20 * 60)).alias("tick"))
    .with_row_index("location")
    .with_columns(pl.col("location") % 384)
).drop(["timestamp_str", "year"])
map_df

location,pbuf_ozone_0,pbuf_ozone_10,pbuf_ozone_20,pbuf_ozone_30,pbuf_ozone_40,pbuf_ozone_50,month,day,seconds,tick
u32,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64
0,4.4335e-7,0.000012,2.3536e-7,8.1375e-8,5.8524e-8,3.5211e-8,1,1,0,0
1,4.3519e-7,0.000012,3.2219e-7,7.8852e-8,5.4014e-8,3.0162e-8,1,1,0,0
2,4.7745e-7,0.000013,1.2123e-7,7.8984e-8,6.1798e-8,3.2987e-8,1,1,0,0
3,4.6701e-7,0.000012,1.5172e-7,8.0751e-8,6.3946e-8,2.7481e-8,1,1,0,0
4,4.3120e-7,0.000012,3.5661e-7,7.7078e-8,5.1578e-8,2.9283e-8,1,1,0,0
…,…,…,…,…,…,…,…,…,…,…
379,5.3979e-7,0.000011,0.000001,1.2733e-7,8.0014e-8,7.3130e-8,1,1,0,0
380,5.4228e-7,0.00001,0.000001,1.2237e-7,7.9764e-8,7.0875e-8,1,1,0,0
381,5.4867e-7,0.000011,9.0636e-7,1.2059e-7,8.3027e-8,7.1162e-8,1,1,0,0


In [9]:
map_df.write_parquet("output/map_hack.parquet")

In [10]:
map_df

location,pbuf_ozone_2,month,day,seconds,tick
u32,f64,i64,i64,i64,i64
0,7.9467e-7,1,1,0,0
1,7.8003e-7,1,1,0,0
2,8.5578e-7,1,1,0,0
3,8.3707e-7,1,1,0,0
4,7.7288e-7,1,1,0,0
…,…,…,…,…,…
379,9.6753e-7,12,31,85200,71
380,9.7199e-7,12,31,85200,71
381,9.8344e-7,12,31,85200,71


In [11]:
len(map_df["pbuf_ozone_2"].unique())

10091520

In [12]:
len(map_df["pbuf_ozone_2"].cast(pl.Float32).unique())

3594961

## 一致しているのかの検証

In [11]:
x1 = np.load("input/ClimSim_low-res/train/0001-02/0001-02-01-00000.npy")
x2 = np.load("input/ClimSim_low-res/train/0002-02/0002-02-01-00000.npy")

In [13]:
train_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/train.csv", n_rows=1)
train_df.columns[1:].index("pbuf_ozone_0")

376

In [12]:
x1[:10, 377:388] == x2[:10, 377:388]

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]])

In [35]:
x2[:10, 377:378]

array([[4.50361359e-07],
       [4.42347221e-07],
       [4.83336415e-07],
       [4.73959806e-07],
       [4.38636056e-07],
       [4.37787253e-07],
       [4.68321652e-07],
       [4.65993386e-07],
       [4.38666593e-07],
       [4.40487285e-07]])